# Evaluate Model

### Load Packages

In [7]:
import os
import time

### Setup

In [16]:
WORKSPACE_PATH = "workspace"
DATA_PATH = os.path.join(WORKSPACE_PATH, "data")
PRE_TRAINED_MODELS_PATH = os.path.join(WORKSPACE_PATH, "pre_trained_models")

MODELS_PATH = os.path.join(WORKSPACE_PATH, "models")
BASE_MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711"
MODEL_NAME = "efficientdet_d1_coco17_tpu-32"

PIPELINE_CONFIG_PATH = os.path.join(MODELS_PATH, MODEL_NAME, "pipeline.config")

EPOCHS = 10
BATCH_SIZE = 4
LEARNING_RATE =  1e-3
USE_AUGMENTATION = False

COMBINATION_NAME = f"epochs_{EPOCHS}-batch_size_{BATCH_SIZE}-learning_rate_{LEARNING_RATE}-aug_{USE_AUGMENTATION}"

In [17]:
!python workspace/model_main_tf2.py \
    --pipeline_config_path=workspace/models/{MODEL_NAME}/{COMBINATION_NAME}/pipeline.config \
    --model_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --checkpoint_dir=workspace/models/{MODEL_NAME}/{COMBINATION_NAME} \
    --eval_timeout=0 \
    --alsologtostderr

/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/jovyan/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase3GetEPNS_15OpKern